In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import sys
import matplotlib.pyplot as plt
import random
import time
#from nxsim import BaseNetworkAgent
from datetime import datetime
import collections
from collections import defaultdict
import copy
import math
nx.__version__

'2.2'

In [2]:
# Set some parameters to apply to all plots. These can be overridden
# in each plot if desired
import matplotlib
# Plot size to 14" x 7"
matplotlib.rc('figure', figsize = (24, 12))
# Font size to 14
matplotlib.rc('font', size = 20)
# Add grid lines
matplotlib.rc('axes', grid = False)
# Set backgound color to white
matplotlib.rc('axes', facecolor = 'white')

In [3]:
read_data = pd.read_csv("pnas.1713568115.sd01.txt") #read the data file, no cleaning required
read_data = read_data[read_data['trial'] == 1]  #take the data for the 1st trial only
data = read_data
data.head(10)
#print(len(data))
#type(data)



data['duration'] = data['end'] - data['begin']#store the diff btw begin and end to get total time for each trans. in ms
data['duration'] = data['duration']/1000.0 #change 'ms' to 's'
#data['time'] = data['time'].map(lambda x: [x]) #convert the time attr to list as as there are multiple iteractions between two bees
data.head()

,trial,id1,id2,begin,end,duration
0,1,201,219,1.373105e+12,1.373105e+12,2.000
1,1,201,219,1.373109e+12,1.373109e+12,42.000
2,1,201,242,1.373119e+12,1.373119e+12,2.000
3,1,201,242,1.373119e+12,1.373119e+12,2.999
4,1,201,255,1.372958e+12,1.372958e+12,41.999


In [4]:
data['begin'] = data['begin']/1000.0 #change 'ms' to 's'
data['day'] = data['begin'].map(lambda x: int(datetime.utcfromtimestamp(x).strftime('%d')))
data.head(10)

,trial,id1,id2,begin,end,duration,day
0,1,201,219,1.373105e+09,1.373105e+12,2.000,6
1,1,201,219,1.373109e+09,1.373109e+12,42.000,6
2,1,201,242,1.373119e+09,1.373119e+12,2.000,6
3,1,201,242,1.373119e+09,1.373119e+12,2.999,6
4,1,201,255,1.372958e+09,1.372958e+12,41.999,4
5,1,201,259,1.372937e+09,1.372937e+12,15.000,4
6,1,201,259,1.373041e+09,1.373041e+12,3.000,5
7,1,201,259,1.373240e+09,1.373240e+12,8.000,7
8,1,201,262,1.373360e+09,1.373360e+12,27.999,9
9,1,201,263,1.373178e+09,1.373178e+12,2.000,7


In [5]:
data['UTC'] = data['begin'].map(lambda x: datetime.utcfromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S'))
data['hours'] = data['begin'].map(lambda x: int(datetime.utcfromtimestamp(x).strftime('%H')))
data = data.sort_values('begin')
data.head(20)

,trial,id1,id2,begin,end,duration,day,UTC,hours
102932,1,612,1697,1.372829e+09,1.372829e+12,3.000,3,2013-07-03 05:29:18,5
138807,1,740,829,1.372829e+09,1.372829e+12,2.000,3,2013-07-03 05:29:23,5
225447,1,1138,1305,1.372829e+09,1.372829e+12,5.999,3,2013-07-03 05:29:32,5
11636,1,267,876,1.372829e+09,1.372829e+12,11.000,3,2013-07-03 05:29:41,5
126121,1,701,912,1.372829e+09,1.372829e+12,10.000,3,2013-07-03 05:29:41,5
119604,1,667,1460,1.372829e+09,1.372829e+12,17.000,3,2013-07-03 05:29:49,5
189002,1,938,1322,1.372829e+09,1.372829e+12,2.000,3,2013-07-03 05:29:52,5
243670,1,1248,1657,1.372829e+09,1.372829e+12,4.000,3,2013-07-03 05:30:18,5
234946,1,1205,1593,1.372829e+09,1.372829e+12,16.999,3,2013-07-03 05:30:25,5
21359,1,288,643,1.372829e+09,1.372829e+12,2.000,3,2013-07-03 05:30:25,5


In [6]:
data_day_3 = data[data['day'] == 3]
len(data_day_3)

30889

In [7]:
data_day_3['duration'] = data_day_3['duration'].map(lambda x: [x]) #convert the begin attr to list as as there are multiple iteractions between two bees
data_day_3['begin'] = data_day_3['begin'].map(lambda x: [x]) #convert the begin attr to list as as there are multiple iteractions between two bees

/Users/apple/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/apple/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [8]:
data_day_3.head(20)

,trial,id1,id2,begin,end,duration,day,UTC,hours
102932,1,612,1697,[1372829358.609],1.372829e+12,[3.0],3,2013-07-03 05:29:18,5
138807,1,740,829,[1372829363.609],1.372829e+12,[2.0],3,2013-07-03 05:29:23,5
225447,1,1138,1305,[1372829372.609],1.372829e+12,[5.999],3,2013-07-03 05:29:32,5
11636,1,267,876,[1372829381.608],1.372829e+12,[11.0],3,2013-07-03 05:29:41,5
126121,1,701,912,[1372829381.608],1.372829e+12,[10.0],3,2013-07-03 05:29:41,5
119604,1,667,1460,[1372829389.608],1.372829e+12,[17.0],3,2013-07-03 05:29:49,5
189002,1,938,1322,[1372829392.608],1.372829e+12,[2.0],3,2013-07-03 05:29:52,5
243670,1,1248,1657,[1372829418.608],1.372829e+12,[4.0],3,2013-07-03 05:30:18,5
234946,1,1205,1593,[1372829425.608],1.372829e+12,[16.999],3,2013-07-03 05:30:25,5
21359,1,288,643,[1372829425.608],1.372829e+12,[2.0],3,2013-07-03 05:30:25,5


In [9]:
data_day_3 = data_day_3.groupby(['id1','id2']).agg({'begin': 'sum', 'duration':'sum'})

In [10]:
data_day_3.head(10)

begin       duration
id1 id2                                                 
201 268                  [1372872921.921]          [5.0]
    272  [1372847419.324, 1372847462.323]  [26.999, 2.0]
    351                   [1372888861.67]          [2.0]
    357                  [1372889999.652]          [2.0]
    363  [1372831883.569, 1372831895.569]     [3.0, 2.0]
    376                  [1372841882.411]          [2.0]
    380                  [1372895570.564]          [2.0]
    393                  [1372865316.041]          [3.0]
    439                  [1372883191.759]          [2.0]
    444                  [1372839146.454]          [3.0]

In [11]:
data_day_3 = data_day_3.reset_index() #reset index after group by as there are some none values for column id1

In [12]:
data_day_3.head(200)

,id1,id2,begin,duration
0,201,268,[1372872921.921],[5.0]
1,201,272,"[1372847419.324, 1372847462.323]","[26.999, 2.0]"
2,201,351,[1372888861.67],[2.0]
3,201,357,[1372889999.652],[2.0]
4,201,363,"[1372831883.569, 1372831895.569]","[3.0, 2.0]"
5,201,376,[1372841882.411],[2.0]
6,201,380,[1372895570.564],[2.0]
7,201,393,[1372865316.041],[3.0]
8,201,439,[1372883191.759],[2.0]
9,201,444,[1372839146.454],[3.0]


In [13]:
G = make_graph(data_day_3)
print(degrees)
sum_of_edges = sum(degrees.values())
mean_degree = sum_of_edges/len(degrees)
print(mean_degree)

NameError: name 'make_graph' is not defined

#### Infinite tau value - add all the durations

In [14]:
data_day_3['sum_duration'] = data_day_3['duration'].map(lambda x: sum(x))

In [15]:
data_day_3.head(200)

,id1,id2,begin,duration,sum_duration
0,201,268,[1372872921.921],[5.0],5.000
1,201,272,"[1372847419.324, 1372847462.323]","[26.999, 2.0]",28.999
2,201,351,[1372888861.67],[2.0],2.000
3,201,357,[1372889999.652],[2.0],2.000
4,201,363,"[1372831883.569, 1372831895.569]","[3.0, 2.0]",5.000
5,201,376,[1372841882.411],[2.0],2.000
6,201,380,[1372895570.564],[2.0],2.000
7,201,393,[1372865316.041],[3.0],3.000
8,201,439,[1372883191.759],[2.0],2.000
9,201,444,[1372839146.454],[3.0],3.000


In [23]:
tau_values = [30,60,90,120] #in seconds
data_30 = data_day_3.copy()
for index,row in data_30.iterrows():
    if len(row[2]) > 1:
        index_list = []
        any_name = []
        for i in range(0,len(row[2]) -1 ): #begin column
            time_diff = row[2][i+1] - row[2][i]
            if time_diff <= 30:
                #index_list.append(i)
                #print(index)
                #print("i=",i)
                any_name.append(row[3][i] + row[3][i+1])
                #print(row[2])
                #print("time dur = ", row[3])
                #row[3][i+1] = row[3][i] + row[3][i+1] 
                #row[3].pop(i)
                #row[2].pop(i)
            else:
                any_name.append(row[3][i])
        print(row[3],any_name)
       
        data_30.at[index,'duration'] = any_name
        #row[3] = any_name
        

[26.999, 2.0] [26.999]
[3.0, 2.0] [5.0]
[3.0, 2.0] [5.0]
[42.0, 11.999, 18.999] [42.0, 30.997999999999998]
[8.0, 3.0] [11.0]
[2.0, 2.0] [4.0]
[2.0, 3.0] [5.0]
[6.0, 5.0, 2.999] [11.0, 7.9990000000000006]
[7.0, 1.999, 4.0, 2.0] [8.999, 5.9990000000000006, 6.0]
[14.999, 4.0, 4.0, 3.0] [18.999000000000002, 8.0, 7.0]
[16.999, 2.0] [16.999]
[3.999, 2.0, 3.0] [5.9990000000000006, 5.0]
[15.0, 9.0] [24.0]
[3.0, 2.0] [3.0]
[2.0, 2.0] [4.0]
[3.0, 3.0] [6.0]
[3.0, 6.0] [9.0]
[5.0, 6.0] [11.0]
[2.0, 2.0] [4.0]
[2.0, 4.0] [6.0]
[3.0, 3.0] [6.0]
[2.0, 3.0, 6.0, 3.0] [5.0, 9.0, 9.0]
[19.999, 3.0] [22.999]
[3.0, 3.0] [3.0]
[1.999, 2.0] [1.999]
[12.0, 7.0, 7.999, 7.0] [19.0, 14.998999999999999, 14.998999999999999]
[4.0, 5.0] [4.0]
[2.0, 5.0] [7.0]
[2.0, 6.0, 6.0] [8.0, 12.0]
[2.0, 2.0] [2.0]
[13.999, 14.0, 17.0] [27.999000000000002, 31.0]
[6.0, 4.0, 2.0, 8.0] [10.0, 6.0, 10.0]
[18.0, 3.0] [21.0]
[17.0, 6.0] [23.0]
[16.0, 5.0, 2.0, 4.0, 8.0] [21.0, 7.0, 6.0, 12.0]
[5.0, 2.0, 2.0] [7.0, 4.0]
[3.0, 2.0] [

[10.0, 9.0] [19.0]
[2.0, 2.0] [2.0]
[4.0, 3.0, 13.0, 6.0, 2.0, 2.0] [7.0, 3.0, 19.0, 6.0, 4.0]
[4.0, 2.0] [6.0]
[4.0, 2.0] [6.0]
[6.0, 2.0] [8.0]
[2.0, 5.0] [7.0]
[3.0, 3.0] [6.0]
[2.0, 2.0] [4.0]
[8.0, 3.0, 10.0, 3.0, 4.0] [11.0, 13.0, 13.0, 7.0]
[10.0, 6.999, 2.0] [16.999, 8.998999999999999]
[3.0, 7.0, 3.0, 5.0, 2.0] [10.0, 10.0, 8.0, 7.0]
[2.0, 4.0] [2.0]
[2.0, 4.0] [6.0]
[6.0, 2.0] [8.0]
[2.0, 2.0] [4.0]
[45.999, 3.0] [45.999]
[2.0, 2.0] [4.0]
[5.0, 4.0, 4.0, 2.0] [9.0, 8.0, 4.0]
[17.0, 3.0] [17.0]
[2.0, 2.0] [2.0]
[17.0, 2.0] [17.0]
[6.0, 10.999, 7.0, 3.0, 4.0, 2.0] [16.999000000000002, 17.999000000000002, 10.0, 7.0, 6.0]
[3.0, 2.0] [5.0]
[2.0, 10.999, 3.999, 24.0] [12.999, 10.999, 27.999]
[3.0, 2.0] [5.0]
[2.0, 11.0] [13.0]
[2.0, 2.0] [4.0]
[4.0, 3.0] [7.0]
[4.0, 2.0, 2.0] [6.0, 4.0]
[3.0, 2.0] [3.0]
[8.0, 4.0, 7.0] [8.0, 4.0]
[11.0, 2.0] [13.0]
[43.0, 2.0] [43.0]
[5.0, 5.0] [10.0]
[2.0, 2.0] [4.0]
[2.0, 9.0] [2.0]
[3.0, 2.0, 2.0] [5.0, 4.0]
[2.0, 7.0] [9.0]
[3.0, 10.0] [13.0]
[4

[8.0, 5.0] [8.0]
[20.0, 5.0, 6.999] [20.0, 11.998999999999999]
[1.999, 7.0, 4.0, 4.0] [8.999, 11.0, 8.0]
[3.0, 2.0] [5.0]
[2.0, 2.0] [4.0]
[6.0, 2.0] [8.0]
[4.0, 9.0, 21.999, 2.0, 2.0, 4.0] [13.0, 30.999, 21.999, 4.0, 6.0]
[4.0, 2.0] [4.0]
[6.0, 2.0] [8.0]
[6.0, 6.0, 10.999, 15.0] [12.0, 16.999000000000002, 25.999000000000002]
[9.0, 3.0, 6.999] [12.0, 9.998999999999999]
[2.0, 2.0, 12.999] [4.0, 14.999]
[4.0, 2.0] [4.0]
[8.0, 5.0] [8.0]
[2.0, 2.0] [4.0]
[2.0, 10.999] [2.0]
[4.0, 2.0] [4.0]
[18.0, 3.0, 4.0] [21.0, 7.0]
[15.999, 4.0] [15.999]
[5.999, 3.0] [8.998999999999999]
[2.0, 3.0] [2.0]
[2.0, 2.0] [4.0]
[13.0, 2.0] [15.0]
[18.0, 2.0, 6.0] [20.0, 8.0]
[2.0, 4.0] [2.0]
[4.0, 5.0, 3.0] [9.0, 8.0]
[15.0, 6.0] [21.0]
[2.0, 3.0, 7.0, 2.0] [5.0, 10.0, 9.0]
[16.999, 2.0] [18.999]
[13.0, 5.0] [18.0]
[4.0, 11.0] [4.0]
[5.0, 2.999] [7.9990000000000006]
[21.0, 4.0] [25.0]
[12.0, 3.0, 2.0] [15.0, 3.0]
[33.999, 7.0] [33.999]
[2.0, 2.0] [4.0]
[32.0, 7.0] [32.0]
[14.999, 6.0, 10.0] [14.999, 16.0]
[7

[2.0, 4.0] [6.0]
[5.0, 2.0] [5.0]
[8.0, 11.0] [19.0]
[3.0, 4.0, 2.0] [7.0, 4.0]
[2.0, 2.0] [4.0]
[6.0, 5.0, 6.0] [6.0, 5.0]
[2.0, 2.0, 2.0] [2.0, 2.0]
[7.0, 2.0] [7.0]
[5.0, 3.0] [5.0]
[2.0, 2.0] [2.0]
[3.0, 2.0] [5.0]
[3.0, 2.0] [5.0]
[4.0, 2.0] [4.0]
[3.0, 3.0] [6.0]
[6.0, 12.0] [18.0]
[4.0, 3.0, 2.0] [7.0, 3.0]
[4.0, 3.0] [7.0]
[25.0, 2.0, 3.0] [27.0, 5.0]
[4.0, 3.0] [7.0]
[5.999, 15.0] [20.999]
[2.0, 5.0] [7.0]
[3.0, 2.0, 3.0, 3.999] [5.0, 5.0, 6.9990000000000006]
[5.999, 41.0] [46.999]
[5.0, 2.0] [7.0]
[7.0, 4.0] [11.0]
[19.0, 4.0, 2.0, 2.0, 2.0] [19.0, 6.0, 4.0, 4.0]
[19.0, 2.0] [21.0]
[2.0, 4.0, 2.0, 5.0] [6.0, 6.0, 7.0]
[6.0, 2.0, 2.0, 4.0] [8.0, 4.0, 6.0]
[2.0, 3.0, 10.0, 2.0, 9.0, 2.0] [5.0, 13.0, 12.0, 11.0, 11.0]
[2.0, 6.0, 4.0, 3.999] [8.0, 10.0, 7.9990000000000006]
[6.999, 7.0] [13.998999999999999]
[10.999, 3.0, 2.0, 3.0, 8.0] [13.999, 5.0, 5.0, 11.0]
[14.0, 5.0] [19.0]
[2.0, 2.0] [2.0]
[2.0, 2.0] [2.0]
[2.0, 6.0] [8.0]
[24.0, 9.0, 3.0] [33.0, 12.0]
[4.0, 5.0] [9.0]
[4.0,

[3.0, 2.0] [5.0]
[2.0, 3.0] [2.0]
[11.0, 2.0] [13.0]
[12.0, 2.0] [14.0]
[8.0, 2.0] [10.0]
[4.0, 8.0] [4.0]
[2.0, 2.0] [2.0]
[12.0, 2.0] [14.0]
[5.0, 13.999, 2.0, 8.0] [18.999000000000002, 15.999, 10.0]
[4.0, 2.0] [6.0]
[4.0, 3.0] [4.0]
[3.0, 2.0] [5.0]
[6.0, 2.0] [6.0]
[11.0, 3.0, 4.0] [14.0, 7.0]
[12.0, 2.0] [14.0]
[21.0, 3.0, 4.0] [24.0, 3.0]
[20.0, 2.0, 3.0] [22.0, 5.0]
[11.0, 3.0, 2.999] [14.0, 5.9990000000000006]
[2.0, 2.0] [2.0]
[5.0, 5.0, 7.0] [10.0, 12.0]
[2.0, 8.0] [10.0]
[2.0, 12.0] [14.0]
[2.0, 2.0] [2.0]
[1.999, 2.0, 4.0] [1.999, 2.0]
[2.0, 2.0] [4.0]
[9.999, 2.0] [9.999]
[3.0, 2.0] [3.0]
[11.0, 32.0] [11.0]
[4.0, 2.0] [6.0]
[39.0, 2.0, 4.0, 3.0] [39.0, 6.0, 7.0]
[5.0, 3.0] [8.0]
[14.0, 7.0] [21.0]
[8.999, 2.0] [8.999]
[41.999, 5.0] [41.999]
[2.0, 4.0, 5.0] [6.0, 9.0]
[3.0, 4.0] [7.0]
[20.0, 2.0] [22.0]
[2.0, 3.0] [2.0]
[12.999, 3.0] [15.999]
[11.999, 6.0] [17.999000000000002]
[3.0, 4.0, 4.0, 5.0] [7.0, 8.0, 9.0]
[2.0, 2.0] [4.0]
[8.0, 14.0] [8.0]
[7.0, 2.0, 5.999] [9.0, 7.

[2.0, 24.0] [2.0]
[8.0, 5.0] [8.0]
[12.0, 2.0, 2.0] [14.0, 2.0]
[6.0, 16.0, 3.0] [22.0, 19.0]
[7.0, 1.999] [7.0]
[3.0, 4.0] [7.0]
[17.0, 1.999, 2.0, 26.999] [18.999, 3.999, 2.0]
[17.0, 3.0] [20.0]
[7.0, 2.0] [9.0]
[2.0, 3.0, 3.0] [5.0, 6.0]
[9.0, 3.0] [12.0]
[2.0, 3.0, 3.0] [5.0, 6.0]
[15.0, 3.0, 2.0] [15.0, 5.0]
[17.0, 2.0] [19.0]
[28.999, 5.0, 2.0, 3.0, 2.999] [28.999, 7.0, 2.0, 5.9990000000000006]
[3.0, 2.0] [5.0]
[3.0, 3.0] [6.0]
[6.0, 4.999, 6.0, 10.0] [10.998999999999999, 10.998999999999999, 16.0]
[10.0, 2.0] [10.0]
[2.0, 2.0] [2.0]
[3.0, 3.999] [3.0]
[9.0, 2.0, 2.0, 2.0] [11.0, 4.0, 4.0]
[10.999, 5.0] [15.999]
[5.0, 5.0] [10.0]
[2.0, 2.0] [4.0]
[25.0, 5.0, 2.0, 4.0] [25.0, 7.0, 6.0]
[2.0, 7.0] [2.0]
[4.0, 3.0] [7.0]
[3.001, 3.0] [6.0009999999999994]
[3.0, 2.0] [5.0]
[2.0, 4.0] [6.0]
[2.0, 17.0, 2.0] [19.0, 19.0]
[5.0, 5.0, 6.0] [10.0, 11.0]
[17.999, 2.0, 6.0, 2.0, 6.0] [19.999, 8.0, 8.0, 8.0]
[2.0, 2.0, 3.0] [2.0, 5.0]
[3.0, 2.0] [3.0]
[7.0, 1.999] [8.999]
[2.0, 5.0] [2.0]
[5.0,

[3.0, 5.0] [3.0]
[25.999, 3.0, 2.0, 14.0, 2.0, 2.0, 2.0] [25.999, 5.0, 16.0, 16.0, 4.0, 4.0]
[11.0, 2.0] [13.0]
[8.0, 2.0, 2.0] [10.0, 2.0]
[5.0, 3.999] [8.999]
[6.0, 2.0] [6.0]
[4.999, 4.0] [8.998999999999999]
[3.0, 3.0, 10.0] [6.0, 13.0]
[3.0, 23.0] [3.0]
[3.0, 3.0, 2.0, 2.0, 3.0, 4.0, 20.0, 6.0] [6.0, 5.0, 4.0, 5.0, 7.0, 24.0, 26.0]
[5.0, 2.0] [7.0]
[2.0, 1.999, 3.0] [3.999, 4.9990000000000006]
[14.999, 7.0] [21.999000000000002]
[2.0, 2.0] [4.0]
[2.0, 17.0] [2.0]
[7.0, 4.0] [11.0]
[2.0, 5.0, 6.0] [7.0, 11.0]
[2.0, 2.0] [2.0]
[3.0, 3.0] [6.0]
[4.999, 2.0] [4.999]
[3.0, 7.0] [3.0]
[23.0, 2.999] [25.999]
[3.0, 4.0, 2.0, 4.0, 2.0] [7.0, 6.0, 6.0, 6.0]
[2.0, 3.0, 3.0, 2.0] [2.0, 6.0, 5.0]
[4.0, 2.0] [6.0]
[3.0, 2.0] [5.0]
[2.0, 2.0] [2.0]
[4.0, 2.0, 10.0] [6.0, 12.0]
[6.0, 2.0, 4.0, 7.999] [6.0, 6.0, 11.998999999999999]
[2.0, 6.0, 2.0] [8.0, 8.0]
[3.0, 3.0, 4.0] [6.0, 7.0]
[8.0, 2.0, 2.0, 7.0] [10.0, 4.0, 9.0]
[2.0, 2.0] [4.0]
[2.0, 2.0] [4.0]
[2.0, 2.0] [4.0]
[15.0, 3.0] [15.0]
[2.0, 2.

[6.0, 5.0] [11.0]
[2.0, 2.0] [4.0]
[32.0, 5.0, 2.0, 2.999] [32.0, 7.0, 4.9990000000000006]
[3.0, 2.0, 1.999, 6.999, 2.0] [5.0, 3.999, 8.998, 8.998999999999999]
[30.999, 2.0, 5.0] [30.999, 7.0]
[14.0, 3.0] [14.0]
[2.0, 1.999, 3.0, 2.0] [3.999, 4.9990000000000006, 5.0]
[7.999, 2.0] [9.998999999999999]
[2.0, 2.0] [4.0]
[7.0, 2.0] [9.0]
[2.0, 4.0, 10.999, 11.0, 8.0] [2.0, 14.999, 21.999000000000002, 19.0]
[3.0, 3.0, 3.0, 2.0] [6.0, 6.0, 3.0]
[13.0, 6.0, 2.0] [19.0, 8.0]
[2.0, 2.0] [4.0]
[2.0, 3.0] [2.0]
[3.0, 2.0] [5.0]
[2.0, 4.0, 16.0, 2.0] [2.0, 20.0, 16.0]
[3.0, 2.0] [5.0]
[1.999, 5.0, 3.0] [6.9990000000000006, 8.0]
[4.0, 4.0] [8.0]
[3.0, 3.0] [3.0]
[6.0, 1.999] [7.9990000000000006]
[25.999, 10.0, 2.0] [25.999, 12.0]
[2.0, 2.0, 9.0, 2.0] [4.0, 11.0, 11.0]
[2.0, 2.0] [2.0]
[3.0, 8.0] [3.0]
[16.0, 2.0] [16.0]
[3.0, 5.0, 2.0] [8.0, 7.0]
[2.0, 2.0] [4.0]
[4.0, 2.0, 2.0, 2.999] [6.0, 4.0, 4.9990000000000006]
[3.0, 17.999, 10.0] [20.999, 27.999]
[5.0, 2.0] [7.0]
[26.0, 14.0] [26.0]
[2.0, 9.0]

[3.0, 4.0] [7.0]
[5.0, 21.0] [26.0]
[23.0, 7.0] [30.0]
[5.0, 3.0, 5.0, 18.0] [8.0, 8.0, 23.0]
[10.0, 3.0, 2.0, 11.0] [13.0, 5.0, 13.0]
[3.0, 3.0, 3.999] [6.0, 6.9990000000000006]
[6.0, 3.0, 2.0, 2.0] [9.0, 5.0, 4.0]
[14.0, 3.999, 4.0, 4.0] [17.999, 7.9990000000000006, 8.0]
[10.0, 2.0] [12.0]
[3.0, 2.0] [3.0]
[2.0, 4.0] [6.0]
[1.999, 4.0, 3.0, 14.0, 6.0, 4.0, 2.0, 4.0, 2.0, 2.0] [5.9990000000000006, 7.0, 17.0, 20.0, 10.0, 6.0, 6.0, 4.0, 4.0]
[3.0, 31.0, 6.0, 2.0] [34.0, 31.0, 8.0]
[5.0, 30.999, 10.0, 3.0] [5.0, 30.999, 13.0]
[2.0, 3.0, 2.0, 2.0] [5.0, 5.0, 4.0]
[6.0, 23.0] [29.0]
[3.0, 4.0] [7.0]
[32.0, 2.0] [32.0]
[2.0, 3.999] [5.9990000000000006]
[7.999, 6.0, 3.0] [7.999, 9.0]
[16.0, 3.0] [16.0]
[23.999, 2.0, 3.0, 6.0] [23.999, 5.0, 9.0]
[9.0, 3.0] [12.0]
[5.0, 3.0] [5.0]
[28.999, 9.0, 3.0] [28.999, 12.0]
[3.0, 4.0] [7.0]
[2.0, 2.0] [2.0]
[2.0, 4.0] [6.0]
[3.0, 5.0] [8.0]
[5.0, 28.0] [5.0]
[2.0, 2.0] [2.0]
[5.0, 1.999, 3.0] [6.9990000000000006, 4.9990000000000006]
[9.0, 15.0] [24.0]
[

[2.0, 2.0] [4.0]
[5.0, 2.999] [7.9990000000000006]
[3.0, 3.0] [3.0]
[1.999, 2.0, 6.0] [1.999, 8.0]
[2.0, 4.0] [6.0]
[2.0, 5.0] [2.0]
[11.0, 2.0] [13.0]
[2.0, 2.0] [4.0]
[2.0, 3.0, 4.0] [5.0, 3.0]
[9.0, 8.0, 2.0, 5.0, 8.0] [17.0, 10.0, 7.0, 5.0]
[6.0, 6.0, 4.0] [12.0, 10.0]
[2.0, 12.0] [14.0]
[6.0, 2.0, 2.0] [8.0, 4.0]
[4.0, 3.999, 23.0] [7.9990000000000006, 26.999]
[2.0, 20.0, 2.0] [2.0, 22.0]
[6.0, 9.0, 4.0] [15.0, 13.0]
[4.0, 2.0, 2.0, 2.0, 5.0, 3.0] [4.0, 4.0, 4.0, 7.0, 8.0]
[27.0, 2.0, 4.0] [27.0, 6.0]
[3.0, 2.0] [5.0]
[3.0, 6.0] [9.0]
[2.0, 6.0, 2.0, 6.0, 6.0, 3.0, 6.0] [2.0, 8.0, 8.0, 12.0, 9.0, 9.0]
[5.0, 20.0] [25.0]
[10.0, 4.0, 4.0, 5.0] [14.0, 8.0, 9.0]
[5.0, 4.0] [9.0]
[3.0, 3.0] [3.0]
[3.0, 2.0] [5.0]
[9.0, 6.0] [15.0]
[2.0, 2.0] [4.0]
[6.0, 6.999] [6.0]
[2.0, 3.0] [5.0]
[5.0, 2.0] [5.0]
[2.0, 8.0] [10.0]
[4.0, 2.0] [6.0]
[5.999, 3.0, 2.0, 2.0] [8.998999999999999, 5.0, 2.0]
[5.0, 18.0] [23.0]
[7.0, 5.0] [12.0]
[2.0, 2.0] [2.0]
[7.0, 7.0, 6.0] [14.0, 13.0]
[4.0, 2.0, 2.0] [6

In [34]:
print(any_name)

[13.0]


In [24]:
len(data_day_3)

24933

In [47]:
dict_30 = defaultdict(int)
for index,row in data_30.iterrows():
    dict_30[row['id1']] += len(row['duration'])

deg_seq_30 = {}
for k,v in dict_30.items():
    if v in deg_seq_30:
        deg_seq_30[v] += 1
    else:
        deg_seq_30[v] = 1
#dict_30
sum_of_edges = sum(list(dict_30.values()))
mean_degree = sum_of_edges/len(dict_30)
print(mean_degree)    

24.03897254207263


In [36]:
len(dict_30)

1129

In [27]:
G = make_graph(data_30)
degrees = dict(G.degree())
sum_of_edges = sum(degrees.values())
mean_degree = sum_of_edges/len(degrees)
print(mean_degree)

42.84020618556701


In [28]:
data_60 = data_day_3.copy()
for index,row in data_60.iterrows():
    if len(row[2]) >1:
        index_list = []
        any_name = []
        for i in range(0,len(row[2]) -1 ): #begin column
            time_diff = row[2][i+1] - row[2][i]
            if time_diff <= 60:
                #index_list.append(i)
                #print(index)
                #print("i=",i)
                any_name.append(row[3][i] + row[3][i+1])
                #print(row[2])
                #print("time dur = ", row[3])
                #row[3][i+1] = row[3][i] + row[3][i+1] 
                #row[3].pop(i)
                #row[2].pop(i)
            else:
                any_name.append(row[3][i])
        print(row[3],any_name)
       
        data_60.at[index,'duration'] = any_name
        #row[3] = any_name

[26.999, 2.0] [28.999]
[3.0, 2.0] [5.0]
[3.0, 2.0] [5.0]
[42.0, 11.999, 18.999] [53.999, 30.997999999999998]
[8.0, 3.0] [11.0]
[2.0, 2.0] [4.0]
[2.0, 3.0] [5.0]
[6.0, 5.0, 2.999] [11.0, 7.9990000000000006]
[7.0, 1.999, 4.0, 2.0] [8.999, 5.9990000000000006, 6.0]
[14.999, 4.0, 4.0, 3.0] [18.999000000000002, 8.0, 7.0]
[16.999, 2.0] [18.999]
[3.999, 2.0, 3.0] [5.9990000000000006, 5.0]
[15.0, 9.0] [24.0]
[3.0, 2.0] [3.0]
[2.0, 2.0] [4.0]
[3.0, 3.0] [6.0]
[3.0, 6.0] [9.0]
[5.0, 6.0] [11.0]
[2.0, 2.0] [4.0]
[2.0, 4.0] [6.0]
[3.0, 3.0] [6.0]
[2.0, 3.0, 6.0, 3.0] [5.0, 9.0, 9.0]
[19.999, 3.0] [22.999]
[3.0, 3.0] [3.0]
[1.999, 2.0] [3.999]
[12.0, 7.0, 7.999, 7.0] [19.0, 14.998999999999999, 14.998999999999999]
[4.0, 5.0] [4.0]
[2.0, 5.0] [7.0]
[2.0, 6.0, 6.0] [8.0, 12.0]
[2.0, 2.0] [2.0]
[13.999, 14.0, 17.0] [27.999000000000002, 31.0]
[6.0, 4.0, 2.0, 8.0] [10.0, 6.0, 10.0]
[18.0, 3.0] [21.0]
[17.0, 6.0] [23.0]
[16.0, 5.0, 2.0, 4.0, 8.0] [21.0, 7.0, 6.0, 12.0]
[5.0, 2.0, 2.0] [7.0, 4.0]
[3.0, 2.0]

[2.0, 2.0] [4.0]
[8.0, 3.0, 10.0, 3.0, 4.0] [11.0, 13.0, 13.0, 7.0]
[10.0, 6.999, 2.0] [16.999, 8.998999999999999]
[3.0, 7.0, 3.0, 5.0, 2.0] [10.0, 10.0, 8.0, 7.0]
[2.0, 4.0] [2.0]
[2.0, 4.0] [6.0]
[6.0, 2.0] [8.0]
[2.0, 2.0] [4.0]
[45.999, 3.0] [45.999]
[2.0, 2.0] [4.0]
[5.0, 4.0, 4.0, 2.0] [9.0, 8.0, 4.0]
[17.0, 3.0] [17.0]
[2.0, 2.0] [2.0]
[17.0, 2.0] [19.0]
[6.0, 10.999, 7.0, 3.0, 4.0, 2.0] [16.999000000000002, 17.999000000000002, 10.0, 7.0, 6.0]
[3.0, 2.0] [5.0]
[2.0, 10.999, 3.999, 24.0] [12.999, 10.999, 27.999]
[3.0, 2.0] [5.0]
[2.0, 11.0] [13.0]
[2.0, 2.0] [4.0]
[4.0, 3.0] [7.0]
[4.0, 2.0, 2.0] [6.0, 4.0]
[3.0, 2.0] [3.0]
[8.0, 4.0, 7.0] [8.0, 11.0]
[11.0, 2.0] [13.0]
[43.0, 2.0] [43.0]
[5.0, 5.0] [10.0]
[2.0, 2.0] [4.0]
[2.0, 9.0] [2.0]
[3.0, 2.0, 2.0] [5.0, 4.0]
[2.0, 7.0] [9.0]
[3.0, 10.0] [13.0]
[4.0, 2.0] [6.0]
[14.0, 7.0, 2.0, 2.0] [21.0, 9.0, 4.0]
[7.0, 3.0] [10.0]
[21.0, 2.0] [23.0]
[13.0, 2.0] [15.0]
[2.0, 17.0] [2.0]
[2.0, 13.0] [15.0]
[10.0, 4.0] [14.0]
[3.0, 4.0, 4.

[2.0, 3.0, 7.0, 2.0] [5.0, 10.0, 9.0]
[16.999, 2.0] [18.999]
[13.0, 5.0] [18.0]
[4.0, 11.0] [4.0]
[5.0, 2.999] [7.9990000000000006]
[21.0, 4.0] [25.0]
[12.0, 3.0, 2.0] [15.0, 3.0]
[33.999, 7.0] [40.999]
[2.0, 2.0] [4.0]
[32.0, 7.0] [32.0]
[14.999, 6.0, 10.0] [20.999000000000002, 16.0]
[7.0, 6.0] [13.0]
[14.0, 2.0] [16.0]
[3.0, 2.0] [5.0]
[6.0, 5.0] [6.0]
[2.0, 2.0] [4.0]
[4.0, 3.0] [7.0]
[2.0, 2.0] [2.0]
[2.0, 4.0, 4.0, 6.0, 6.0] [6.0, 8.0, 10.0, 12.0]
[5.0, 3.0, 4.0, 5.0, 8.0] [5.0, 7.0, 9.0, 13.0]
[14.0, 6.0] [20.0]
[7.0, 2.0] [9.0]
[10.0, 4.0, 4.0, 13.0, 3.0] [14.0, 8.0, 17.0, 16.0]
[8.0, 5.0] [13.0]
[2.0, 4.0] [6.0]
[2.0, 3.0] [5.0]
[3.0, 7.0] [10.0]
[8.0, 2.0] [10.0]
[5.0, 3.0] [5.0]
[5.0, 2.0, 2.0, 3.0] [7.0, 4.0, 5.0]
[4.0, 5.0] [9.0]
[6.0, 3.0] [9.0]
[2.0, 10.0, 6.999] [12.0, 10.0]
[19.0, 5.0] [24.0]
[4.0, 4.0, 2.0] [8.0, 4.0]
[18.999, 4.0, 4.0] [22.999, 8.0]
[2.0, 2.0] [2.0]
[2.0, 4.0] [6.0]
[4.0, 2.0, 2.0] [6.0, 4.0]
[7.0, 3.0] [10.0]
[2.0, 2.0, 2.0, 7.0] [4.0, 4.0, 9.0]
[4.0

[2.0, 2.0, 3.0] [4.0, 5.0]
[2.0, 4.0] [6.0]
[3.0, 4.0, 6.0] [7.0, 10.0]
[4.0, 10.999] [4.0]
[3.0, 2.0] [5.0]
[2.0, 2.999, 4.0, 3.0] [4.9990000000000006, 6.9990000000000006, 7.0]
[5.0, 3.0, 8.0] [8.0, 11.0]
[2.0, 3.0, 9.0] [5.0, 12.0]
[2.0, 2.0] [4.0]
[2.0, 3.0] [5.0]
[3.0, 11.0, 8.0] [3.0, 11.0]
[2.0, 8.0] [10.0]
[2.0, 3.0, 16.0] [5.0, 19.0]
[6.0, 2.0, 3.0] [6.0, 5.0]
[5.0, 14.0] [19.0]
[1.999, 2.0] [1.999]
[3.0, 1.999] [4.9990000000000006]
[23.0, 4.0, 2.0, 4.0, 4.999, 2.0, 3.0] [27.0, 6.0, 6.0, 8.998999999999999, 6.999, 5.0]
[13.0, 5.0] [13.0]
[2.0, 2.0] [2.0]
[7.0, 3.0] [7.0]
[15.999, 3.0] [18.999000000000002]
[25.999, 10.999] [25.999]
[21.999, 2.0] [23.999]
[11.0, 5.0, 7.0] [16.0, 12.0]
[4.999, 2.0] [4.999]
[15.0, 2.0] [17.0]
[6.0, 1.999, 4.0] [7.9990000000000006, 5.9990000000000006]
[8.0, 5.0] [13.0]
[5.0, 2.0, 4.0] [7.0, 6.0]
[3.0, 4.0] [7.0]
[4.0, 2.0, 3.0] [6.0, 5.0]
[8.0, 2.0] [10.0]
[3.0, 2.0] [3.0]
[2.0, 2.0] [4.0]
[3.0, 8.0] [11.0]
[2.0, 8.999] [10.999]
[2.0, 2.0, 10.0] [4.0

[2.0, 10.0] [2.0]
[5.0, 10.0, 2.0] [5.0, 12.0]
[9.0, 14.999, 4.0, 4.0, 4.0, 2.0] [23.999000000000002, 18.999000000000002, 8.0, 8.0, 6.0]
[19.0, 2.0] [21.0]
[50.999, 4.0, 10.999] [50.999, 14.999]
[4.0, 12.0] [16.0]
[15.0, 2.0, 7.0, 2.0] [17.0, 9.0, 7.0]
[8.0, 2.0] [10.0]
[14.999, 3.0] [17.999000000000002]
[2.0, 14.0, 2.0, 2.0, 2.0, 4.0, 2.0] [16.0, 16.0, 4.0, 4.0, 6.0, 4.0]
[5.0, 2.0] [5.0]
[34.999, 2.0] [34.999]
[6.0, 13.0] [6.0]
[9.0, 2.0] [11.0]
[3.0, 2.0] [3.0]
[6.0, 4.0] [10.0]
[4.0, 9.0] [4.0]
[2.0, 2.0] [2.0]
[2.0, 9.0, 6.0, 4.0, 13.0] [11.0, 9.0, 10.0, 17.0]
[14.0, 2.0, 6.999] [16.0, 8.998999999999999]
[2.0, 3.0, 5.0] [5.0, 8.0]
[2.0, 2.0] [4.0]
[2.0, 4.0, 3.0, 8.0, 2.0] [6.0, 7.0, 11.0, 10.0]
[2.0, 29.0, 23.0] [2.0, 52.0]
[28.999, 3.0, 3.0, 2.0, 2.0, 3.0] [31.999, 6.0, 3.0, 4.0, 5.0]
[5.0, 14.999] [19.999000000000002]
[2.0, 14.999, 9.0, 13.0] [16.999000000000002, 23.999000000000002, 22.0]
[2.0, 8.999, 3.0] [10.999, 11.999]
[33.0, 4.0] [37.0]
[3.999, 36.999] [3.999]
[4.0, 4.0, 3

[3.0, 5.0] [8.0]
[4.0, 2.0] [6.0]
[3.0, 3.0, 2.0, 3.0, 6.0, 3.0] [6.0, 5.0, 5.0, 9.0, 9.0]
[4.0, 2.0] [4.0]
[5.0, 12.0, 3.0, 2.0] [17.0, 15.0, 5.0]
[5.0, 3.0] [8.0]
[2.0, 5.0] [7.0]
[18.0, 2.0] [20.0]
[14.999, 2.0, 8.0, 3.0, 2.0] [16.999000000000002, 10.0, 11.0, 5.0]
[9.0, 2.0] [11.0]
[16.0, 3.0, 2.0] [19.0, 5.0]
[3.0, 14.0] [17.0]
[3.0, 8.0, 3.0, 8.0, 5.0, 5.0, 7.0] [11.0, 11.0, 11.0, 13.0, 10.0, 12.0]
[2.0, 2.0] [4.0]
[7.0, 2.0] [9.0]
[9.0, 7.0, 7.0] [16.0, 14.0]
[4.999, 3.0, 3.0, 3.0] [7.999, 6.0, 6.0]
[2.0, 12.999] [2.0]
[3.0, 8.0] [11.0]
[29.999, 2.0, 4.0, 3.0] [31.999, 6.0, 7.0]
[4.0, 12.999, 2.0] [16.999000000000002, 14.999]
[3.0, 2.0] [5.0]
[4.999, 10.0] [4.999]
[2.999, 2.0] [4.9990000000000006]
[17.0, 3.0] [20.0]
[2.0, 3.0, 3.0] [2.0, 3.0]
[2.0, 14.0] [2.0]
[5.999, 8.0, 3.0] [13.998999999999999, 11.0]
[6.0, 7.0] [13.0]
[33.999, 5.0] [33.999]
[7.0, 2.0] [7.0]
[24.0, 15.0] [39.0]
[3.0, 2.0] [5.0]
[2.0, 4.0] [2.0]
[2.0, 2.0] [4.0]
[19.0, 2.999] [21.999]
[4.0, 4.0] [8.0]
[3.0, 3.0

[2.0, 3.0, 3.0, 2.0] [5.0, 6.0, 5.0]
[4.0, 2.0] [6.0]
[3.0, 2.0] [5.0]
[2.0, 2.0] [2.0]
[4.0, 2.0, 10.0] [6.0, 12.0]
[6.0, 2.0, 4.0, 7.999] [8.0, 6.0, 11.998999999999999]
[2.0, 6.0, 2.0] [8.0, 8.0]
[3.0, 3.0, 4.0] [6.0, 7.0]
[8.0, 2.0, 2.0, 7.0] [10.0, 4.0, 9.0]
[2.0, 2.0] [4.0]
[2.0, 2.0] [4.0]
[2.0, 2.0] [4.0]
[15.0, 3.0] [15.0]
[2.0, 2.0] [2.0]
[15.0, 2.0, 13.0] [17.0, 15.0]
[9.0, 7.0, 3.0, 4.0] [16.0, 10.0, 7.0]
[57.999, 2.0] [57.999]
[8.999, 3.0, 5.0, 4.0] [11.999, 8.0, 9.0]
[18.0, 61.999] [18.0]
[2.0, 2.0, 2.0] [4.0, 4.0]
[2.0, 2.0] [4.0]
[3.0, 2.0] [5.0]
[11.0, 2.0] [13.0]
[4.0, 3.0, 5.999, 3.0] [7.0, 8.998999999999999, 8.998999999999999]
[10.0, 6.0, 9.0, 5.0, 2.0] [16.0, 15.0, 9.0, 7.0]
[43.999, 4.0] [43.999]
[11.999, 3.0, 2.0, 5.0] [14.999, 5.0, 2.0]
[2.0, 3.0, 6.0, 2.0] [2.0, 9.0, 8.0]
[3.0, 10.0] [13.0]
[3.0, 2.0] [5.0]
[6.0, 5.999, 3.0] [11.998999999999999, 8.998999999999999]
[2.0, 8.0] [10.0]
[14.0, 11.0, 3.0] [25.0, 14.0]
[4.0, 2.0] [6.0]
[2.0, 2.0] [4.0]
[8.999, 1.999] [

[24.999, 3.0, 5.0, 2.0] [27.999, 8.0, 7.0]
[3.0, 4.0, 3.0] [7.0, 7.0]
[12.0, 3.0] [15.0]
[2.0, 7.0, 13.0] [9.0, 20.0]
[6.0, 2.0] [8.0]
[7.0, 3.0, 4.0, 2.0] [10.0, 7.0, 6.0]
[3.0, 2.0] [3.0]
[12.0, 6.0] [18.0]
[28.0, 4.0, 3.0] [32.0, 7.0]
[11.0, 3.0, 5.0] [11.0, 8.0]
[1.999, 2.001] [4.0]
[5.999, 4.0, 6.0] [9.998999999999999, 10.0]
[2.0, 2.0] [4.0]
[8.0, 4.0] [8.0]
[21.0, 8.0, 11.0, 4.0, 6.0, 11.0] [29.0, 19.0, 15.0, 10.0, 17.0]
[2.0, 2.0, 7.999, 4.0] [4.0, 9.998999999999999, 11.998999999999999]
[2.0, 2.0, 4.0, 6.0] [4.0, 6.0, 10.0]
[8.0, 2.0, 3.0] [10.0, 5.0]
[2.0, 8.0, 6.0] [10.0, 14.0]
[7.0, 12.0, 3.0] [7.0, 12.0]
[5.0, 3.0, 5.0, 2.0] [8.0, 8.0, 7.0]
[2.0, 2.0] [4.0]
[10.0, 7.0, 8.0] [10.0, 15.0]
[4.0, 4.0, 3.0] [4.0, 7.0]
[28.0, 11.0, 4.0] [39.0, 11.0]
[10.999, 5.0] [15.999]
[29.999, 2.0] [29.999]
[2.0, 2.0] [4.0]
[2.0, 18.0] [2.0]
[8.0, 4.0] [12.0]
[3.0, 14.999] [17.999000000000002]
[7.0, 2.0] [7.0]
[4.0, 8.0] [12.0]
[13.999, 6.0] [19.999000000000002]
[2.0, 1.999] [3.999]
[7.0, 4.0]

[10.0, 9.999, 4.0] [19.999000000000002, 13.999]
[19.0, 3.0, 4.0, 4.0] [22.0, 7.0, 8.0]
[5.0, 8.0] [13.0]
[3.0, 4.0] [7.0]
[5.0, 21.0] [26.0]
[23.0, 7.0] [30.0]
[5.0, 3.0, 5.0, 18.0] [8.0, 8.0, 23.0]
[10.0, 3.0, 2.0, 11.0] [13.0, 5.0, 13.0]
[3.0, 3.0, 3.999] [6.0, 6.9990000000000006]
[6.0, 3.0, 2.0, 2.0] [9.0, 5.0, 4.0]
[14.0, 3.999, 4.0, 4.0] [17.999, 7.9990000000000006, 8.0]
[10.0, 2.0] [12.0]
[3.0, 2.0] [3.0]
[2.0, 4.0] [6.0]
[1.999, 4.0, 3.0, 14.0, 6.0, 4.0, 2.0, 4.0, 2.0, 2.0] [5.9990000000000006, 7.0, 17.0, 20.0, 10.0, 6.0, 6.0, 4.0, 4.0]
[3.0, 31.0, 6.0, 2.0] [34.0, 37.0, 8.0]
[5.0, 30.999, 10.0, 3.0] [5.0, 40.998999999999995, 13.0]
[2.0, 3.0, 2.0, 2.0] [5.0, 5.0, 4.0]
[6.0, 23.0] [29.0]
[3.0, 4.0] [7.0]
[32.0, 2.0] [34.0]
[2.0, 3.999] [5.9990000000000006]
[7.999, 6.0, 3.0] [7.999, 9.0]
[16.0, 3.0] [16.0]
[23.999, 2.0, 3.0, 6.0] [25.999, 5.0, 9.0]
[9.0, 3.0] [12.0]
[5.0, 3.0] [5.0]
[28.999, 9.0, 3.0] [37.998999999999995, 12.0]
[3.0, 4.0] [7.0]
[2.0, 2.0] [2.0]
[2.0, 4.0] [6.0]
[3

[2.0, 14.0] [2.0]
[2.0, 12.0, 4.0, 2.999] [14.0, 16.0, 6.9990000000000006]
[5.0, 4.0] [5.0]
[3.0, 3.0, 2.0, 5.0, 5.0, 17.0] [6.0, 5.0, 7.0, 10.0, 22.0]
[3.999, 8.0, 6.0] [11.999, 14.0]
[2.0, 2.0, 5.0, 2.0, 7.0] [4.0, 7.0, 7.0, 9.0]
[16.999, 2.0] [18.999]
[5.0, 5.0, 6.0] [5.0, 11.0]
[3.0, 12.999] [3.0]
[2.0, 3.0] [5.0]
[11.0, 2.0] [13.0]
[6.0, 7.0] [6.0]
[2.0, 2.0, 2.0] [4.0, 2.0]
[3.0, 2.0, 3.0] [5.0, 5.0]
[5.0, 5.0] [10.0]
[12.0, 3.0, 3.0, 3.999] [15.0, 3.0, 6.9990000000000006]
[11.0, 3.0] [14.0]
[17.0, 6.0] [23.0]
[39.0, 5.0] [44.0]
[2.0, 2.0] [4.0]
[2.0, 2.0] [4.0]
[4.0, 3.0, 3.0] [7.0, 6.0]
[2.0, 2.0, 3.0] [2.0, 2.0]
[21.0, 11.999, 11.0, 2.0, 3.0, 2.0] [32.999, 22.999000000000002, 13.0, 5.0, 3.0]
[34.999, 5.0, 4.0] [34.999, 5.0]
[2.0, 4.0] [6.0]
[2.0, 3.0] [5.0]
[2.0, 3.0] [2.0]
[2.0, 2.0, 11.0, 2.0, 2.0] [4.0, 13.0, 13.0, 4.0]
[2.0, 2.0] [4.0]
[3.0, 4.0] [7.0]
[5.0, 3.0, 2.0] [8.0, 5.0]
[7.999, 3.0] [7.999]
[5.0, 2.0, 3.0, 2.0, 3.0] [7.0, 5.0, 5.0, 5.0]
[5.0, 3.0] [8.0]
[9.0, 2.0,

In [48]:
dict_60 = defaultdict(int)
for index,row in data_60.iterrows():
    dict_60[row['id1']] += len(row['duration'])

deg_seq_60 = {}
for k,v in dict_60.items():
    if v in deg_seq_60:
        deg_seq_60[v] += 1
    else:
        deg_seq_60[v] = 1
    
#dict_30
sum_of_edges = sum(dict_60.values())
mean_degree = sum_of_edges/len(dict_60)
print(mean_degree) 

24.03897254207263


In [43]:
sum(data_30['duration'] != data_60['duration'])

382

In [42]:
data_90 = data_day_3.copy()
for index,row in data_90.iterrows():
    if len(row[2]) >1:
        index_list = []
        any_name = []
        for i in range(0,len(row[2]) -1 ): #begin column
            time_diff = row[2][i+1] - row[2][i]
            if time_diff <= 90:
                #index_list.append(i)
                #print(index)
                #print("i=",i)
                any_name.append(row[3][i] + row[3][i+1])
                #print(row[2])
                #print("time dur = ", row[3])
                #row[3][i+1] = row[3][i] + row[3][i+1] 
                #row[3].pop(i)
                #row[2].pop(i)
            else:
                any_name.append(row[3][i])
        print(row[3],any_name)
       
        data_90.at[index,'duration'] = any_name
        #row[3] = any_name

[26.999, 2.0] [28.999]
[3.0, 2.0] [5.0]
[3.0, 2.0] [5.0]
[42.0, 11.999, 18.999] [53.999, 30.997999999999998]
[8.0, 3.0] [11.0]
[2.0, 2.0] [4.0]
[2.0, 3.0] [5.0]
[6.0, 5.0, 2.999] [11.0, 7.9990000000000006]
[7.0, 1.999, 4.0, 2.0] [8.999, 5.9990000000000006, 6.0]
[14.999, 4.0, 4.0, 3.0] [18.999000000000002, 8.0, 7.0]
[16.999, 2.0] [18.999]
[3.999, 2.0, 3.0] [5.9990000000000006, 5.0]
[15.0, 9.0] [24.0]
[3.0, 2.0] [3.0]
[2.0, 2.0] [4.0]
[3.0, 3.0] [6.0]
[3.0, 6.0] [9.0]
[5.0, 6.0] [11.0]
[2.0, 2.0] [4.0]
[2.0, 4.0] [6.0]
[3.0, 3.0] [6.0]
[2.0, 3.0, 6.0, 3.0] [5.0, 9.0, 9.0]
[19.999, 3.0] [22.999]
[3.0, 3.0] [3.0]
[1.999, 2.0] [3.999]
[12.0, 7.0, 7.999, 7.0] [19.0, 14.998999999999999, 14.998999999999999]
[4.0, 5.0] [4.0]
[2.0, 5.0] [7.0]
[2.0, 6.0, 6.0] [8.0, 12.0]
[2.0, 2.0] [2.0]
[13.999, 14.0, 17.0] [27.999000000000002, 31.0]
[6.0, 4.0, 2.0, 8.0] [10.0, 6.0, 10.0]
[18.0, 3.0] [21.0]
[17.0, 6.0] [23.0]
[16.0, 5.0, 2.0, 4.0, 8.0] [21.0, 7.0, 6.0, 12.0]
[5.0, 2.0, 2.0] [7.0, 4.0]
[3.0, 2.0]

[2.0, 2.0] [2.0]
[17.0, 2.0] [19.0]
[6.0, 10.999, 7.0, 3.0, 4.0, 2.0] [16.999000000000002, 17.999000000000002, 10.0, 7.0, 6.0]
[3.0, 2.0] [5.0]
[2.0, 10.999, 3.999, 24.0] [12.999, 14.998000000000001, 27.999]
[3.0, 2.0] [5.0]
[2.0, 11.0] [13.0]
[2.0, 2.0] [4.0]
[4.0, 3.0] [7.0]
[4.0, 2.0, 2.0] [6.0, 4.0]
[3.0, 2.0] [3.0]
[8.0, 4.0, 7.0] [8.0, 11.0]
[11.0, 2.0] [13.0]
[43.0, 2.0] [43.0]
[5.0, 5.0] [10.0]
[2.0, 2.0] [4.0]
[2.0, 9.0] [2.0]
[3.0, 2.0, 2.0] [5.0, 4.0]
[2.0, 7.0] [9.0]
[3.0, 10.0] [13.0]
[4.0, 2.0] [6.0]
[14.0, 7.0, 2.0, 2.0] [21.0, 9.0, 4.0]
[7.0, 3.0] [10.0]
[21.0, 2.0] [23.0]
[13.0, 2.0] [15.0]
[2.0, 17.0] [2.0]
[2.0, 13.0] [15.0]
[10.0, 4.0] [14.0]
[3.0, 4.0, 4.0, 2.0, 2.0] [7.0, 8.0, 6.0, 2.0]
[2.0, 2.0] [2.0]
[2.0, 5.0] [7.0]
[4.0, 3.0, 3.0, 2.0, 4.0, 3.0] [7.0, 6.0, 5.0, 6.0, 7.0]
[2.0, 2.0] [4.0]
[2.0, 7.0, 5.0] [9.0, 12.0]
[2.0, 2.0] [4.0]
[24.0, 3.0, 6.0, 9.999] [27.0, 9.0, 15.999]
[6.0, 3.0] [9.0]
[10.0, 3.0] [13.0]
[10.0, 2.0] [12.0]
[2.0, 3.0] [5.0]
[2.0, 4.0, 4.

[17.0, 6.0] [17.0]
[2.0, 8.0] [2.0]
[2.0, 2.0] [4.0]
[2.0, 2.0, 2.0] [4.0, 4.0]
[2.999, 3.0] [5.9990000000000006]
[5.999, 2.0] [7.999]
[4.0, 4.0] [8.0]
[4.0, 2.0, 5.999, 69.999] [6.0, 2.0, 5.999]
[3.0, 2.0, 9.0] [5.0, 11.0]
[2.0, 4.0] [6.0]
[3.0, 11.0, 2.0, 4.0, 2.0] [3.0, 13.0, 6.0, 6.0]
[3.0, 23.999] [26.999]
[20.0, 2.0, 10.0] [22.0, 12.0]
[5.0, 9.0] [5.0]
[4.0, 5.0] [9.0]
[2.0, 2.0] [4.0]
[4.0, 5.0] [9.0]
[4.0, 2.0, 6.0, 4.0] [6.0, 8.0, 10.0]
[3.0, 11.0] [3.0]
[2.0, 11.0] [13.0]
[2.0, 1.999, 2.0, 4.0] [3.999, 3.999, 6.0]
[2.0, 2.0] [4.0]
[20.0, 2.0] [22.0]
[31.999, 2.0, 4.0] [33.998999999999995, 6.0]
[5.0, 2.0, 2.0] [7.0, 2.0]
[3.0, 3.0, 6.0, 9.0] [6.0, 9.0, 15.0]
[12.0, 4.0] [16.0]
[4.0, 6.0, 3.0] [10.0, 9.0]
[15.0, 2.0] [17.0]
[2.0, 5.0, 3.0] [7.0, 8.0]
[2.0, 5.0, 2.0] [7.0, 7.0]
[3.0, 2.0] [5.0]
[4.0, 9.999, 15.0, 3.0, 4.0, 5.0] [13.999, 24.999000000000002, 18.0, 7.0, 9.0]
[3.0, 4.0] [3.0]
[3.0, 1.999] [4.9990000000000006]
[2.0, 3.0] [5.0]
[13.0, 5.0] [18.0]
[15.0, 4.0] [19.0]
[1

[4.0, 8.0] [12.0]
[2.0, 6.0, 5.0, 5.0] [8.0, 11.0, 10.0]
[3.0, 2.0] [5.0]
[7.999, 3.0, 17.0, 2.0, 7.0] [10.998999999999999, 20.0, 19.0, 9.0]
[2.0, 9.0, 4.0, 23.999, 2.0] [2.0, 13.0, 27.999, 25.999]
[2.0, 2.0, 6.0, 8.0] [4.0, 8.0, 14.0]
[2.0, 18.0] [20.0]
[12.999, 3.0, 13.0, 4.0] [15.999, 16.0, 17.0]
[8.0, 7.0] [15.0]
[4.0, 5.0] [9.0]
[12.0, 6.999] [18.999]
[17.0, 2.0] [19.0]
[12.0, 9.0] [21.0]
[5.999, 12.0] [5.999]
[18.0, 1.999, 7.0, 7.0] [19.999, 8.999, 14.0]
[4.0, 1.999, 2.0] [4.0, 1.999]
[2.999, 2.0] [4.9990000000000006]
[2.0, 3.0, 3.0] [5.0, 6.0]
[16.0, 6.0, 3.0] [22.0, 9.0]
[2.0, 12.0] [14.0]
[32.999, 10.0, 3.0, 2.0] [42.999, 13.0, 5.0]
[6.0, 2.0] [8.0]
[4.0, 6.0] [10.0]
[5.0, 2.0] [7.0]
[13.0, 4.0] [17.0]
[5.0, 2.999] [7.9990000000000006]
[4.0, 3.0] [4.0]
[20.0, 15.999] [20.0]
[3.0, 2.0] [5.0]
[23.999, 4.0] [27.999]
[6.0, 3.0, 2.0] [9.0, 5.0]
[5.0, 7.0, 2.0, 2.0, 11.0, 2.0] [12.0, 9.0, 4.0, 2.0, 11.0]
[13.0, 11.999] [24.999000000000002]
[21.0, 12.999] [21.0]
[2.0, 2.0] [4.0]
[7.0

[4.0, 2.0] [6.0]
[7.0, 2.0, 8.0] [9.0, 10.0]
[4.999, 2.0] [6.999]
[4.0, 3.0] [7.0]
[2.0, 5.0] [7.0]
[13.0, 6.0] [19.0]
[13.0, 3.0, 4.0] [16.0, 7.0]
[8.0, 2.0, 1.999] [10.0, 3.999]
[24.999, 14.0] [38.998999999999995]
[7.0, 8.0] [7.0]
[23.0, 2.0] [25.0]
[2.0, 9.0] [2.0]
[14.0, 5.999] [19.999]
[2.0, 1.999, 2.0, 5.0] [2.0, 3.999, 2.0]
[7.0, 4.0, 6.999, 4.0] [11.0, 10.998999999999999, 10.998999999999999]
[6.0, 2.0] [6.0]
[2.0, 6.0] [8.0]
[15.0, 17.0] [32.0]
[5.0, 3.0] [5.0]
[2.0, 2.0, 2.0] [4.0, 4.0]
[7.0, 3.0, 2.0, 3.0] [10.0, 5.0, 5.0]
[2.0, 5.0] [2.0]
[2.0, 13.0] [2.0]
[14.0, 3.0] [17.0]
[40.999, 4.0] [44.999]
[3.0, 2.0, 16.999] [5.0, 18.999]
[29.999, 2.0] [31.999]
[13.0, 7.0] [20.0]
[2.0, 2.0] [4.0]
[2.0, 2.0] [4.0]
[3.0, 2.0, 2.0] [3.0, 2.0]
[2.0, 2.0, 2.0] [4.0, 2.0]
[23.0, 3.0, 2.0, 5.0] [26.0, 5.0, 7.0]
[2.999, 6.0, 3.0] [8.999, 9.0]
[9.999, 5.0] [9.999]
[2.0, 63.999, 8.0] [2.0, 71.999]
[2.0, 2.0] [2.0]
[2.0, 2.0] [2.0]
[2.0, 12.0] [2.0]
[2.0, 2.0] [4.0]
[8.0, 3.0] [8.0]
[20.0, 2.0,

[3.0, 2.0] [3.0]
[7.0, 2.0] [9.0]
[2.0, 36.999] [2.0]
[2.0, 9.0, 17.999, 11.0, 2.0] [11.0, 26.999, 28.999, 13.0]
[4.0, 3.0] [4.0]
[28.999, 6.0, 2.0, 2.0] [34.998999999999995, 8.0, 4.0]
[3.0, 3.0] [6.0]
[5.0, 2.0] [7.0]
[6.999, 2.0, 9.0, 6.0, 7.0, 2.0, 8.0] [8.998999999999999, 11.0, 15.0, 13.0, 9.0, 10.0]
[2.0, 2.0] [2.0]
[53.0, 5.0, 2.0, 4.0] [58.0, 7.0, 6.0]
[3.0, 3.0, 5.0] [6.0, 8.0]
[3.0, 7.0] [10.0]
[1.999, 15.0] [1.999]
[23.999, 2.0] [23.999]
[3.0, 2.0] [5.0]
[6.0, 3.0, 2.999, 2.0, 6.0, 4.0, 2.0] [9.0, 5.9990000000000006, 4.9990000000000006, 2.0, 10.0, 6.0]
[3.0, 10.0] [3.0]
[12.0, 3.0] [12.0]
[5.0, 9.0, 11.0] [5.0, 20.0]
[5.0, 2.0, 2.0] [7.0, 4.0]
[17.0, 3.001] [20.001]
[18.999, 9.0] [27.999]
[3.0, 2.999, 26.0] [5.9990000000000006, 2.999]
[11.0, 4.0, 4.0] [15.0, 4.0]
[4.0, 3.0] [7.0]
[2.0, 3.0] [5.0]
[14.999, 3.0] [17.999000000000002]
[2.0, 2.0] [4.0]
[4.0, 5.0] [9.0]
[7.0, 5.0, 8.0] [12.0, 13.0]
[2.0, 2.0] [4.0]
[2.0, 23.999] [2.0]
[2.0, 9.0, 8.0] [11.0, 17.0]
[3.0, 2.0] [5.0]
[

[3.0, 6.0] [3.0]
[3.0, 2.0] [3.0]
[10.0, 2.0] [10.0]
[5.0, 3.0] [5.0]
[4.0, 2.0] [6.0]
[3.0, 4.0] [7.0]
[3.0, 4.0] [7.0]
[5.0, 2.0] [7.0]
[7.0, 2.0] [9.0]
[4.0, 4.0] [4.0]
[22.0, 2.0] [22.0]
[2.0, 2.0] [4.0]
[5.0, 2.0, 4.0] [7.0, 6.0]
[3.0, 2.0] [5.0]
[3.0, 22.0, 2.0, 4.0, 21.0] [25.0, 24.0, 2.0, 25.0]
[3.0, 4.0] [7.0]
[4.0, 6.0, 3.0, 5.0, 2.0, 3.0] [4.0, 9.0, 8.0, 7.0, 5.0]
[3.0, 9.0] [12.0]
[3.0, 2.0] [5.0]
[11.0, 5.0] [16.0]
[16.0, 13.0, 4.0] [29.0, 17.0]
[12.0, 4.0] [16.0]
[2.0, 7.0, 3.0, 3.0, 2.0] [9.0, 10.0, 6.0, 5.0]
[5.0, 2.0, 5.0] [7.0, 7.0]
[5.0, 6.0] [11.0]
[7.0, 2.0, 2.0] [9.0, 4.0]
[4.0, 2.0, 96.999] [6.0, 2.0]
[4.0, 11.0] [15.0]
[5.0, 2.0] [5.0]
[17.0, 2.0] [19.0]
[19.0, 3.0] [22.0]
[3.0, 3.0] [6.0]
[14.0, 2.0] [16.0]
[6.0, 14.0] [20.0]
[49.999, 2.0] [51.999]
[2.0, 1.999] [2.0]
[9.0, 13.0] [22.0]
[10.0, 4.0] [14.0]
[2.0, 2.0] [4.0]
[2.0, 3.0] [5.0]
[2.0, 2.0, 4.0] [4.0, 6.0]
[3.0, 2.0, 4.0, 3.0] [5.0, 6.0, 7.0]
[2.0, 2.0, 6.0] [4.0, 8.0]
[5.0, 4.0, 8.999] [9.0, 12.999]
[8

[16.999, 2.0] [18.999]
[6.0, 7.0] [13.0]
[14.0, 2.999] [16.999]
[2.0, 4.0, 4.0] [6.0, 8.0]
[3.0, 4.0] [7.0]
[5.0, 3.0] [8.0]
[4.0, 2.0] [6.0]
[2.0, 2.0] [4.0]
[17.0, 7.0] [24.0]
[2.0, 3.0] [5.0]
[3.0, 11.0, 8.0] [14.0, 11.0]
[16.999, 3.999, 2.0] [20.997999999999998, 3.999]
[4.0, 5.0] [9.0]
[17.0, 4.0] [17.0]
[2.0, 2.0] [4.0]
[2.0, 3.0] [5.0]
[2.0, 14.0] [16.0]
[3.0, 5.0, 3.0] [8.0, 8.0]
[18.0, 2.0] [20.0]
[3.0, 3.0] [6.0]
[2.0, 2.0] [4.0]
[3.0, 6.0, 6.0] [9.0, 12.0]
[3.0, 1.999] [3.0]
[2.0, 2.0, 3.0, 6.0, 2.0, 5.0] [4.0, 5.0, 9.0, 8.0, 2.0]
[3.0, 2.0] [3.0]
[2.0, 2.0, 2.0, 1.999] [4.0, 4.0, 3.999]
[3.0, 2.0] [5.0]
[35.0, 2.0, 2.0] [37.0, 4.0]
[2.0, 2.0, 3.0] [4.0, 5.0]
[2.0, 2.0] [4.0]
[2.0, 3.0, 4.0] [5.0, 7.0]
[3.0, 2.0] [5.0]
[6.0, 2.0, 3.0] [8.0, 5.0]
[3.0, 28.0, 6.0] [3.0, 34.0]
[19.0, 4.0, 8.0, 7.0] [23.0, 12.0, 15.0]
[3.0, 5.999, 3.0] [8.998999999999999, 8.998999999999999]
[3.0, 2.0] [5.0]
[13.0, 3.0, 2.0] [16.0, 5.0]
[11.0, 3.0, 3.0] [14.0, 6.0]
[7.999, 4.0] [11.998999999999999

[3.0, 3.0] [6.0]
[3.0, 2.0] [3.0]
[9.999, 4.0, 4.0, 10.0, 5.0, 4.0, 3.0, 3.0] [13.999, 8.0, 14.0, 15.0, 9.0, 4.0, 3.0]
[17.0, 3.0] [20.0]
[4.0, 2.0, 2.0] [6.0, 4.0]
[13.0, 4.0, 9.0, 7.0, 5.0, 3.0, 3.999] [17.0, 13.0, 16.0, 12.0, 8.0, 6.9990000000000006]
[5.0, 2.0] [7.0]
[7.0, 2.0, 2.0] [9.0, 4.0]
[3.0, 3.0, 3.0, 3.0] [6.0, 6.0, 6.0]
[2.0, 4.0] [2.0]
[13.0, 5.0] [18.0]
[13.0, 3.0] [16.0]
[2.0, 4.0] [6.0]
[2.0, 2.0] [2.0]
[3.999, 2.0] [5.9990000000000006]
[2.0, 7.999] [9.998999999999999]
[8.0, 11.0] [8.0]
[7.0, 5.0] [12.0]
[6.0, 7.0] [13.0]
[30.0, 2.0, 5.0] [32.0, 7.0]
[10.0, 2.001] [12.001]
[2.0, 6.999] [8.998999999999999]
[137.998, 3.0] [137.998]
[2.0, 2.0] [2.0]
[3.0, 7.0, 2.0] [10.0, 9.0]
[6.0, 7.999, 2.0] [13.998999999999999, 9.998999999999999]
[5.0, 2.0, 3.0] [7.0, 5.0]
[4.0, 5.0, 2.0] [9.0, 7.0]
[2.0, 2.0, 2.0] [4.0, 4.0]
[2.0, 2.0] [2.0]
[2.999, 9.0, 3.0] [11.999, 12.0]
[6.0, 2.0, 4.0] [8.0, 6.0]
[3.0, 2.0] [5.0]
[3.0, 6.0, 4.0, 2.0, 6.0, 4.0, 4.0, 2.0, 2.0, 3.0] [9.0, 10.0, 6.0,

[2.0, 2.0] [2.0]
[2.0, 9.0] [2.0]
[2.0, 2.0, 4.0] [4.0, 6.0]
[2.0, 3.0, 6.0, 7.0] [2.0, 9.0, 13.0]
[2.0, 4.0] [2.0]
[3.0, 4.0] [7.0]
[2.0, 5.0] [7.0]
[9.0, 2.0] [11.0]
[6.0, 3.0] [6.0]
[6.0, 3.0] [9.0]
[3.0, 3.0] [6.0]
[10.0, 4.0, 3.0, 20.0, 2.0, 6.0] [14.0, 7.0, 3.0, 22.0, 8.0]
[2.0, 4.0] [6.0]
[2.0, 7.0] [2.0]
[5.0, 17.0] [5.0]
[3.0, 4.0, 3.0] [7.0, 7.0]
[4.0, 8.0] [12.0]
[2.0, 2.0] [4.0]
[9.0, 2.0] [9.0]
[3.0, 2.0, 13.0] [5.0, 15.0]
[1.999, 2.0, 2.0, 5.0, 3.0] [3.999, 4.0, 7.0, 8.0]
[2.0, 4.0, 5.0] [6.0, 9.0]
[16.0, 7.0, 27.999, 5.0] [23.0, 34.998999999999995, 32.998999999999995]
[12.0, 2.0, 5.0, 15.0] [14.0, 7.0, 5.0]
[3.0, 16.0] [3.0]
[7.0, 4.0] [11.0]
[2.0, 2.0] [4.0]
[2.0, 6.0] [8.0]
[15.0, 5.0, 3.0, 4.999, 22.0, 8.0] [20.0, 8.0, 7.999, 4.999, 30.0]
[28.999, 2.0] [30.999]
[2.0, 2.0] [2.0]
[3.0, 5.0, 7.0, 2.0] [8.0, 12.0, 9.0]
[28.0, 9.0, 16.999] [37.0, 25.999]
[5.0, 3.0] [5.0]
[5.0, 6.0] [11.0]
[14.0, 2.0] [16.0]
[2.0, 5.0] [7.0]
[2.0, 4.0, 2.0, 2.0, 12.0, 6.0, 11.0] [6.0, 6.0, 

In [31]:
data_120 = data_day_3.copy()
for index,row in data_120.iterrows():
    if len(row[2]) >1:
        index_list = []
        any_name = []
        for i in range(0,len(row[2]) -1 ): #begin column
            time_diff = row[2][i+1] - row[2][i]
            if time_diff <= 120:
                #index_list.append(i)
                #print(index)
                #print("i=",i)
                any_name.append(row[3][i] + row[3][i+1])
                #print(row[2])
                #print("time dur = ", row[3])
                #row[3][i+1] = row[3][i] + row[3][i+1] 
                #row[3].pop(i)
                #row[2].pop(i)
            else:
                any_name.append(row[3][i])
        print(row[3],any_name)
       
        data_120.at[index,'duration'] = any_name
        #row[3] = any_name

[26.999, 2.0] [28.999]
[3.0, 2.0] [5.0]
[3.0, 2.0] [5.0]
[42.0, 11.999, 18.999] [53.999, 30.997999999999998]
[8.0, 3.0] [11.0]
[2.0, 2.0] [4.0]
[2.0, 3.0] [5.0]
[6.0, 5.0, 2.999] [11.0, 7.9990000000000006]
[7.0, 1.999, 4.0, 2.0] [8.999, 5.9990000000000006, 6.0]
[14.999, 4.0, 4.0, 3.0] [18.999000000000002, 8.0, 7.0]
[16.999, 2.0] [18.999]
[3.999, 2.0, 3.0] [5.9990000000000006, 5.0]
[15.0, 9.0] [24.0]
[3.0, 2.0] [3.0]
[2.0, 2.0] [4.0]
[3.0, 3.0] [6.0]
[3.0, 6.0] [9.0]
[5.0, 6.0] [11.0]
[2.0, 2.0] [4.0]
[2.0, 4.0] [6.0]
[3.0, 3.0] [6.0]
[2.0, 3.0, 6.0, 3.0] [5.0, 9.0, 9.0]
[19.999, 3.0] [22.999]
[3.0, 3.0] [3.0]
[1.999, 2.0] [3.999]
[12.0, 7.0, 7.999, 7.0] [19.0, 14.998999999999999, 14.998999999999999]
[4.0, 5.0] [9.0]
[2.0, 5.0] [7.0]
[2.0, 6.0, 6.0] [8.0, 12.0]
[2.0, 2.0] [2.0]
[13.999, 14.0, 17.0] [27.999000000000002, 31.0]
[6.0, 4.0, 2.0, 8.0] [10.0, 6.0, 10.0]
[18.0, 3.0] [21.0]
[17.0, 6.0] [23.0]
[16.0, 5.0, 2.0, 4.0, 8.0] [21.0, 7.0, 6.0, 12.0]
[5.0, 2.0, 2.0] [7.0, 4.0]
[3.0, 2.0]

[12.0, 3.0] [15.0]
[8.0, 2.0, 11.0] [10.0, 13.0]
[12.0, 5.0] [17.0]
[3.999, 2.0] [5.9990000000000006]
[32.0, 2.0] [34.0]
[3.0, 2.0] [3.0]
[2.0, 2.001] [4.0009999999999994]
[2.0, 9.0, 3.0] [11.0, 12.0]
[6.0, 2.0] [8.0]
[4.0, 16.0] [4.0]
[10.0, 9.0] [19.0]
[2.0, 2.0] [2.0]
[4.0, 3.0, 13.0, 6.0, 2.0, 2.0] [7.0, 16.0, 19.0, 6.0, 4.0]
[4.0, 2.0] [6.0]
[4.0, 2.0] [6.0]
[6.0, 2.0] [8.0]
[2.0, 5.0] [7.0]
[3.0, 3.0] [6.0]
[2.0, 2.0] [4.0]
[8.0, 3.0, 10.0, 3.0, 4.0] [11.0, 13.0, 13.0, 7.0]
[10.0, 6.999, 2.0] [16.999, 8.998999999999999]
[3.0, 7.0, 3.0, 5.0, 2.0] [10.0, 10.0, 8.0, 7.0]
[2.0, 4.0] [2.0]
[2.0, 4.0] [6.0]
[6.0, 2.0] [8.0]
[2.0, 2.0] [4.0]
[45.999, 3.0] [45.999]
[2.0, 2.0] [4.0]
[5.0, 4.0, 4.0, 2.0] [9.0, 8.0, 4.0]
[17.0, 3.0] [17.0]
[2.0, 2.0] [2.0]
[17.0, 2.0] [19.0]
[6.0, 10.999, 7.0, 3.0, 4.0, 2.0] [16.999000000000002, 17.999000000000002, 10.0, 7.0, 6.0]
[3.0, 2.0] [5.0]
[2.0, 10.999, 3.999, 24.0] [12.999, 14.998000000000001, 27.999]
[3.0, 2.0] [5.0]
[2.0, 11.0] [13.0]
[2.0, 2.0] 

[2.0, 4.0] [6.0]
[3.0, 3.0] [3.0]
[4.0, 5.0, 7.0] [4.0, 5.0]
[14.0, 8.0, 4.0] [22.0, 8.0]
[14.0, 2.0] [16.0]
[9.0, 2.0] [11.0]
[3.0, 2.0, 2.0] [5.0, 4.0]
[2.0, 2.0] [4.0]
[2.0, 2.0] [4.0]
[37.0, 3.0, 2.0, 2.0, 2.0, 2.0] [40.0, 5.0, 4.0, 2.0, 2.0]
[2.0, 2.0] [2.0]
[2.0, 8.0] [10.0]
[22.0, 7.999, 3.0] [29.999, 10.998999999999999]
[29.0, 12.999] [29.0]
[5.0, 15.0] [20.0]
[3.0, 2.999] [5.9990000000000006]
[7.0, 3.0] [10.0]
[2.0, 3.0] [5.0]
[3.0, 3.0] [6.0]
[17.999, 2.0] [17.999]
[19.0, 12.999, 3.999] [31.999000000000002, 12.999]
[2.0, 3.0] [5.0]
[2.0, 2.0, 2.0] [4.0, 4.0]
[9.0, 4.0] [13.0]
[4.0, 9.0] [13.0]
[4.0, 7.0] [4.0]
[3.0, 2.0] [5.0]
[2.0, 2.0] [4.0]
[3.0, 2.0] [5.0]
[3.0, 8.999, 2.0] [11.999, 10.999]
[7.0, 3.0, 2.0] [10.0, 5.0]
[11.0, 11.0, 3.0, 5.0] [22.0, 14.0, 8.0]
[5.0, 2.0, 3.0] [5.0, 2.0]
[11.0, 4.0] [15.0]
[26.0, 2.0] [26.0]
[4.0, 2.0] [6.0]
[2.0, 5.0] [7.0]
[6.0, 5.0] [11.0]
[5.0, 3.0] [8.0]
[20.999, 2.0] [22.999]
[5.0, 2.0, 4.0, 3.0] [7.0, 6.0, 7.0]
[4.0, 8.0] [12.0]
[2.0,

[2.0, 2.0] [2.0]
[2.0, 6.0] [8.0]
[24.0, 9.0, 3.0] [33.0, 12.0]
[4.0, 5.0] [9.0]
[4.0, 3.0] [7.0]
[5.0, 6.0] [11.0]
[27.999, 3.0, 2.0, 2.0] [30.999, 5.0, 4.0]
[2.0, 11.0, 6.0] [13.0, 17.0]
[3.0, 14.0] [3.0]
[3.0, 5.0] [8.0]
[16.0, 3.0, 9.0, 3.001] [19.0, 12.0, 12.001]
[5.0, 4.0] [9.0]
[9.0, 3.0, 20.0] [12.0, 23.0]
[2.0, 2.0] [4.0]
[15.999, 2.0] [17.999000000000002]
[3.0, 3.0, 4.0] [3.0, 7.0]
[12.0, 9.999, 2.0] [12.0, 11.999]
[23.999, 4.0] [27.999]
[2.0, 2.999] [4.9990000000000006]
[2.0, 2.0] [4.0]
[2.0, 5.0] [7.0]
[3.0, 3.0, 2.0] [3.0, 3.0]
[5.0, 2.0] [5.0]
[11.0, 3.0] [14.0]
[2.0, 4.0, 2.0] [6.0, 6.0]
[9.0, 9.0] [18.0]
[1.999, 4.0] [5.9990000000000006]
[4.0, 2.0, 2.0] [6.0, 4.0]
[2.0, 2.0, 3.0] [4.0, 5.0]
[2.0, 4.0] [6.0]
[3.0, 4.0, 6.0] [7.0, 10.0]
[4.0, 10.999] [4.0]
[3.0, 2.0] [5.0]
[2.0, 2.999, 4.0, 3.0] [4.9990000000000006, 6.9990000000000006, 7.0]
[5.0, 3.0, 8.0] [8.0, 11.0]
[2.0, 3.0, 9.0] [5.0, 12.0]
[2.0, 2.0] [4.0]
[2.0, 3.0] [5.0]
[3.0, 11.0, 8.0] [3.0, 11.0]
[2.0, 8.0] [10

[2.0, 3.0, 4.0] [5.0, 7.0]
[6.0, 4.0, 2.0, 11.0, 2.0, 2.0] [10.0, 4.0, 2.0, 13.0, 4.0]
[4.0, 5.0] [9.0]
[2.0, 2.0] [4.0]
[5.999, 3.0] [5.999]
[45.999, 4.0] [49.999]
[2.0, 4.999] [6.999]
[2.0, 4.0, 2.0] [6.0, 6.0]
[4.0, 30.0] [4.0]
[11.0, 2.0] [11.0]
[2.0, 4.0, 3.0, 8.0] [2.0, 7.0, 11.0]
[3.0, 3.0, 12.0, 2.0] [6.0, 3.0, 14.0]
[19.0, 6.0, 3.0] [25.0, 9.0]
[21.0, 6.0] [27.0]
[2.0, 6.0] [8.0]
[15.0, 6.0, 5.0] [21.0, 11.0]
[2.0, 54.999, 2.0, 3.0, 2.0] [2.0, 56.999, 5.0, 5.0]
[4.0, 2.0] [6.0]
[6.0, 2.0, 2.999] [8.0, 2.0]
[6.999, 4.0] [10.998999999999999]
[2.0, 1.999] [3.999]
[23.0, 2.0] [25.0]
[64.999, 3.0] [67.999]
[4.0, 4.999] [8.998999999999999]
[6.0, 2.0] [8.0]
[37.0, 3.0, 4.0] [40.0, 7.0]
[2.0, 6.0, 8.999] [8.0, 6.0]
[2.0, 3.0, 5.0] [5.0, 8.0]
[3.0, 6.999, 2.0] [9.998999999999999, 8.998999999999999]
[8.0, 14.0, 6.999] [22.0, 20.999]
[2.0, 2.0] [2.0]
[2.0, 8.0] [2.0]
[4.0, 4.0] [8.0]
[2.0, 4.0] [2.0]
[13.0, 2.0, 4.001, 3.0, 2.0] [15.0, 6.001, 7.001, 5.0]
[7.999, 3.0] [10.998999999999999]

[2.0, 2.0] [4.0]
[10.0, 2.0] [10.0]
[2.0, 2.0] [2.0]
[4.0, 1.999] [5.9990000000000006]
[6.0, 2.0] [8.0]
[6.0, 3.0] [9.0]
[3.0, 3.0, 2.0, 2.0] [6.0, 5.0, 4.0]
[2.0, 2.0] [4.0]
[5.0, 4.0, 6.0, 9.0] [9.0, 10.0, 15.0]
[2.0, 6.0] [2.0]
[2.0, 2.0] [4.0]
[7.0, 3.0] [7.0]
[2.0, 2.0] [2.0]
[2.0, 1.999, 2.999] [3.999, 4.998]
[15.0, 2.0] [15.0]
[2.0, 3.0] [5.0]
[2.0, 2.0] [4.0]
[3.0, 3.0, 7.0] [6.0, 10.0]
[15.999, 2.0] [17.999000000000002]
[7.0, 7.0] [7.0]
[4.0, 8.0] [12.0]
[2.0, 3.0] [5.0]
[5.0, 2.0, 2.0, 3.0, 3.0, 6.0, 3.0] [7.0, 4.0, 5.0, 6.0, 9.0, 9.0]
[7.0, 4.999] [7.0]
[20.0, 4.0] [24.0]
[2.0, 4.0, 13.999, 6.0, 2.0, 2.0] [6.0, 17.999000000000002, 19.999000000000002, 8.0, 4.0]
[7.0, 3.0] [10.0]
[13.999, 3.0] [16.999000000000002]
[3.0, 6.0, 22.0] [9.0, 28.0]
[50.999, 23.0, 6.0] [50.999, 29.0]
[2.0, 12.0, 13.0, 4.999] [14.0, 25.0, 17.999]
[2.999, 9.0] [11.999]
[16.0, 2.0] [18.0]
[16.0, 10.0, 5.0] [26.0, 15.0]
[6.0, 4.0] [6.0]
[2.0, 8.0] [2.0]
[6.0, 3.0, 2.0, 2.0] [9.0, 5.0, 4.0]
[2.0, 2.0, 5.0

[25.999, 3.0, 2.0, 14.0, 2.0, 2.0, 2.0] [28.999, 5.0, 16.0, 16.0, 4.0, 4.0]
[11.0, 2.0] [13.0]
[8.0, 2.0, 2.0] [10.0, 2.0]
[5.0, 3.999] [8.999]
[6.0, 2.0] [6.0]
[4.999, 4.0] [8.998999999999999]
[3.0, 3.0, 10.0] [6.0, 13.0]
[3.0, 23.0] [3.0]
[3.0, 3.0, 2.0, 2.0, 3.0, 4.0, 20.0, 6.0] [6.0, 5.0, 4.0, 5.0, 7.0, 24.0, 26.0]
[5.0, 2.0] [7.0]
[2.0, 1.999, 3.0] [3.999, 4.9990000000000006]
[14.999, 7.0] [21.999000000000002]
[2.0, 2.0] [4.0]
[2.0, 17.0] [2.0]
[7.0, 4.0] [11.0]
[2.0, 5.0, 6.0] [7.0, 11.0]
[2.0, 2.0] [4.0]
[3.0, 3.0] [6.0]
[4.999, 2.0] [4.999]
[3.0, 7.0] [3.0]
[23.0, 2.999] [25.999]
[3.0, 4.0, 2.0, 4.0, 2.0] [7.0, 6.0, 6.0, 6.0]
[2.0, 3.0, 3.0, 2.0] [5.0, 6.0, 5.0]
[4.0, 2.0] [6.0]
[3.0, 2.0] [5.0]
[2.0, 2.0] [2.0]
[4.0, 2.0, 10.0] [6.0, 12.0]
[6.0, 2.0, 4.0, 7.999] [8.0, 6.0, 11.998999999999999]
[2.0, 6.0, 2.0] [8.0, 8.0]
[3.0, 3.0, 4.0] [6.0, 7.0]
[8.0, 2.0, 2.0, 7.0] [10.0, 4.0, 9.0]
[2.0, 2.0] [4.0]
[2.0, 2.0] [4.0]
[2.0, 2.0] [4.0]
[15.0, 3.0] [15.0]
[2.0, 2.0] [2.0]
[15.0, 2

[8.0, 2.0, 3.0] [10.0, 5.0]
[2.0, 8.0, 6.0] [10.0, 14.0]
[7.0, 12.0, 3.0] [7.0, 12.0]
[5.0, 3.0, 5.0, 2.0] [8.0, 8.0, 7.0]
[2.0, 2.0] [4.0]
[10.0, 7.0, 8.0] [10.0, 15.0]
[4.0, 4.0, 3.0] [4.0, 7.0]
[28.0, 11.0, 4.0] [39.0, 11.0]
[10.999, 5.0] [15.999]
[29.999, 2.0] [31.999]
[2.0, 2.0] [4.0]
[2.0, 18.0] [2.0]
[8.0, 4.0] [12.0]
[3.0, 14.999] [17.999000000000002]
[7.0, 2.0] [7.0]
[4.0, 8.0] [12.0]
[13.999, 6.0] [19.999000000000002]
[2.0, 1.999] [3.999]
[7.0, 4.0] [11.0]
[6.999, 5.0] [11.998999999999999]
[23.0, 4.999, 4.0] [27.999, 8.998999999999999]
[7.0, 2.0] [7.0]
[2.0, 2.0, 2.0] [4.0, 4.0]
[18.999, 11.0] [29.999]
[12.0, 3.0] [15.0]
[2.0, 3.0] [5.0]
[2.0, 3.0] [5.0]
[2.0, 8.0] [10.0]
[21.0, 6.0] [27.0]
[21.999, 7.0] [28.999]
[2.0, 3.0] [5.0]
[2.0, 3.0] [5.0]
[3.0, 2.0] [3.0]
[3.0, 6.0] [9.0]
[21.0, 14.999] [35.999]
[11.0, 4.0, 5.0] [15.0, 9.0]
[2.0, 23.0, 4.0, 10.0, 2.0] [2.0, 27.0, 14.0, 12.0]
[2.0, 2.0] [4.0]
[2.0, 3.0] [5.0]
[14.0, 6.999] [20.999]
[2.0, 1.999] [2.0]
[14.999, 2.0] [16.

[3.0, 16.0, 4.0, 2.0, 12.0] [19.0, 20.0, 6.0, 14.0]
[2.0, 5.0] [7.0]
[30.999, 2.0] [32.998999999999995]
[2.0, 2.0] [4.0]
[2.0, 2.0, 2.0] [4.0, 4.0]
[4.0, 3.0, 3.0] [4.0, 3.0]
[16.0, 2.0] [16.0]
[4.0, 2.999, 9.0] [6.9990000000000006, 11.999]
[3.0, 2.0] [3.0]
[6.0, 4.0] [10.0]
[7.0, 3.0] [10.0]
[30.999, 2.0, 2.0, 3.0] [32.998999999999995, 4.0, 5.0]
[2.0, 15.0] [17.0]
[3.0, 7.0] [10.0]
[37.999, 3.0, 7.0] [40.999, 10.0]
[7.999, 2.0] [9.998999999999999]
[4.0, 6.999] [10.998999999999999]
[13.0, 9.0, 6.0] [22.0, 15.0]
[10.999, 3.0] [13.999]
[3.0, 2.0, 2.0] [5.0, 4.0]
[4.0, 2.0, 2.0] [6.0, 2.0]
[3.0, 5.0] [8.0]
[88.999, 4.0, 30.0, 6.0] [92.999, 34.0, 36.0]
[25.999, 2.0, 4.0] [27.999, 6.0]
[3.0, 2.0] [3.0]
[2.0, 3.0, 3.0] [5.0, 6.0]
[5.0, 2.0, 2.0] [7.0, 4.0]
[5.0, 2.0, 2.0, 2.0, 3.999] [5.0, 4.0, 4.0, 5.9990000000000006]
[3.0, 3.0] [6.0]
[3.0, 7.0, 3.999, 3.999, 2.0] [10.0, 10.999, 7.998, 5.9990000000000006]
[2.0, 6.999] [8.998999999999999]
[44.0, 7.0] [51.0]
[2.001, 6.0] [8.001]
[16.0, 5.0, 1

[2.0, 2.0] [2.0]
[2.0, 4.0] [6.0]
[17.0, 4.0, 4.0] [21.0, 4.0]
[2.999, 3.0] [5.9990000000000006]
[21.999, 12.0] [33.998999999999995]
[8.0, 2.0] [10.0]
[36.0, 2.0] [38.0]
[3.0, 4.0, 3.0] [7.0, 7.0]
[2.0, 4.0] [2.0]
[2.0, 2.0, 2.0, 4.0] [4.0, 4.0, 6.0]
[2.0, 6.0] [8.0]
[3.0, 2.0, 2.0] [5.0, 4.0]
[17.999, 2.0, 3.0, 4.0] [19.999, 5.0, 7.0]
[2.0, 2.0] [2.0]
[19.0, 2.0] [19.0]
[4.0, 3.0] [7.0]
[4.0, 3.0, 3.999, 4.0, 3.0, 3.0] [7.0, 6.9990000000000006, 7.9990000000000006, 7.0, 6.0]
[3.0, 12.0, 13.0, 4.0] [15.0, 25.0, 17.0]
[25.999, 2.0] [27.999]
[8.0, 4.0] [12.0]
[9.0, 4.0] [13.0]
[3.0, 2.0] [5.0]
[2.0, 3.0] [2.0]
[10.0, 4.999, 7.0, 2.0] [14.998999999999999, 11.998999999999999, 9.0]
[21.0, 9.0, 4.0] [30.0, 13.0]
[2.0, 15.0] [17.0]
[7.0, 2.0] [9.0]
[5.999, 2.0] [7.999]
[2.0, 2.0] [4.0]
[6.0, 4.0, 6.0, 3.0] [10.0, 10.0, 9.0]
[3.001, 4.0] [7.0009999999999994]
[4.0, 2.999] [4.0]
[8.0, 2.0] [10.0]
[2.0, 2.0] [4.0]
[2.0, 2.0] [4.0]
[4.0, 2.0, 2.0, 2.0] [4.0, 2.0, 4.0]
[3.0, 4.0, 2.0, 5.0, 5.0] [7.0

In [32]:
def make_graph(data): #make a graph out of the dataframe structure
    G = nx.Graph() #initialize the graph
    for index, row in data.iterrows():
        #print(row[0],row[1],row[2])
        G.add_edge(row[0],row[1], weight = copy.deepcopy(row[3])) #add node1,node2,weight list iteratively to the graph
    
    return G

In [33]:
for val in tau_values:
    d = defaultdict(int)
    if val == 30:
       
        for index,row in data_30.iterrows():
            d[row[0]] += len(row[3])
    elif val == 60:
        
        for index,row in data_60.iterrows():
            d[row[0]] += len(row[3])
    elif val == 90:
       
        for index,row in data_90.iterrows():
            d[row[0]] += len(row[3])
    else:
    
        for index,row in data_120.iterrows():
            d[row[0]] += len(row[3])
        
   
    #dict_30
    sum_of_edges = sum(d.values())
    mean_degree = sum_of_edges/len(d)
    #sum_of_edges = sum(degrees.values())
    #mean_degree = sum_of_edges/len(d)
    print(mean_degree)

24.03897254207263
24.03897254207263
24.03897254207263
24.03897254207263


In [22]:
data_120 = data_day_3.copy()
cout120 = 0
for index,row in data_120.iterrows():
    if len(row[2]) >1:
        index_list = []
        any_name = []
        for i in range(0,len(row[2]) -1 ): #begin column
            time_diff = row[2][i+1] - row[2][i]
            if time_diff <= 120:
                #index_list.append(i)
                #print(index)
                #print("i=",i)
                cout120 += 1
                any_name.append(row[3][i] + row[3][i+1])
                #print(row[2])
                #print("time dur = ", row[3])
                #row[3][i+1] = row[3][i] + row[3][i+1] 
                #row[3].pop(i)
                #row[2].pop(i)
            else:
                any_name.append(row[3][i])
        #print(row[3],any_name)
       
        data_120.at[index,'duration'] = any_name
        #row[3] = any_name
print(cout120)

5025


In [ ]:
30 4474
60 4875
90 4966
120 5025

In [ ]:
#Redundant code below

In [373]:
tau_values = [30,60,90,120] #in seconds
data_60 = data_day_3.copy()
for index,row in data_60.iterrows():
    if len(row[2]) >1:
        index_list = []
        any_name = []
        for i in range(0,len(row[2]) -1 ): #begin column
            time_diff = row[2][i+1] - row[2][i]
            if time_diff <= 10:
                #index_list.append(i)
                #print(index)
                #print("i=",i)
                any_name.append(row[3][i] + row[3][i+1])
                #print(row[2])
                #print("time dur = ", row[3])
                #row[3][i+1] = row[3][i] + row[3][i+1] 
                #row[3].pop(i)
                #row[2].pop(i)
            else:
                any_name.append(row[3][i])
        #print(row[3],any_name)
       
        data_60.at[index,'duration'] = any_name
        #row[3] = any_name
dict_60 = defaultdict(int)
for index,row in data_60.iterrows():
    dict_60[row[0]] += len(row[3])

#dict_30
sum_of_edges = sum(dict_60.values())
mean_degree = sum_of_edges/len(dict_60)
print(mean_degree)           

24.03897254207263


In [356]:
dict_60

defaultdict(int, {})